<a href="https://colab.research.google.com/github/Rafsan7238/N_Queens_Genetic/blob/main/N-Queens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CSE422 lab: Genetic Algorithm

#### Genetic Algorithm Pseudo code:



**function** GENETIC-ALGORITHM( population, FITNESS-FN) **returns** an individual 
 
> **inputs:** population- a set of individuals/chromosomes; FITNESS-FN- a function that measures the fitness of an individual

>**repeat** 
new_population $\leftarrow$ empty set 
>>**for** $i=1$ **to** size ($ population$) **do**
$$
\begin{array}{l}
x \leftarrow \text { RANDOM-SELECTION }(\text { population, FITNESS-FN }) \\
y \leftarrow \text { RANDOM-SELECTION }(\text { population, FITNESS-FN }) \\
child  \leftarrow \operatorname{CROSSOVER}(x, y)
\end{array}
$$
>>>**if** (some_random_number < mutation_threshold) **then** child$\leftarrow$ MUTATE ( child ) 

>>>add child to new_population 

>>population $\leftarrow$ new_population 

>**until** some individual is fit enough, or enough time has elapsed

>**return** the best individual in population, according to FITNESS-FN
 


### Skeleton Code:

### Importing libraries

In [ ]:
import numpy as np
import random
import copy

### Fitness function

In [ ]:
def findIndividualFitness(individual, n):
  '''
  given an indivual chromosome, calculates the fitness score and returns it.
  '''

  max_fitness = (n*(n-1))//2

  # find horizontal collisions

  horizontal_collision = 0
  for queen in individual:
    count = 0
    for another_queen in individual:
      if queen == another_queen:
        count += 1
    horizontal_collision += (count-1)

  horizontal_collision /= 2

  # initiate diagonal space on both sides for up and down diagonals

  left_diag_space = np.zeros(2*n)
  right_diag_space = np.zeros(2*n)

  # find overlapping positions for diagonal collisions

  for index in range(0, n):
    left_diag_space[index + individual[index] - 1] += 1
    right_diag_space[n - index + individual[index] - 2] += 1

  diag_collision = 0

  # find diagonal collisions

  for index in range(0, 2*n-1):
        count = 0
        if left_diag_space[index] > 1:
            count += left_diag_space[index]-1
        if right_diag_space[index] > 1:
            count += right_diag_space[index]-1
        diag_collision += (count / (n-abs(index-n+1)))

  individual_fitness_score = max_fitness-horizontal_collision-diag_collision

  return individual_fitness_score


In [ ]:
def fitness(population, n):

  '''calculates the fitness score of each
     of the individuals in the population
     
     returns a 1D numpy array: index referring to 
     ith individual in population, and value referring 
     to the fitness score.'''

  fitness_score = []

  for individual_index in range(0,len(population)):
    fitness_score.append(findIndividualFitness(population[individual_index], n))


  return fitness_score

### Random Selection function

This built-in function might help to create the weighted random selection:

`numpy.random.choice(a, size, replace, p)` 

`p` are the weights of the individuals- value between 0 and 1; refers to the probability of each individual being selected.

`a` is the array

`size` how many samples to return

`replace = True`

In [ ]:
def select(population, fit):
  ''' take input:  population and fit
      fit contains fitness values of each of the individuals 
      in the population  
      
      return:  one individual randomly giving
      more weight to ones which have high fitness score'''

  prob = []

  # calculate the weights

  for fitness_score in fit:
    prob.append(fitness_score/sum(fit))

  population_index = []

  for i in range(0, len(population)):
    population_index.append(i)

  random_individual_index = random.choices(population_index, weights=prob, k=1)[0]
  random_individual = population[random_individual_index]

  
  return random_individual

### Crossover function


**function** CROSSOVER $(x, y)$ **returns** an individual 

>**inputs**: $x, y$  which are the parent individuals

>$n \leftarrow \mathrm{LENGTH}(x) ; c \leftarrow$ random number from 1 to $n$ 

>**return** APPEND(SUBSTRING $(x, 1, c),$ SUBSTRING $(y, c+1, n))$

In [ ]:
def crossover(x, y):
  '''take input: 2 parents - x, y. 
     Generate a random crossover point. 
     Append first half of x with second 
     half of y to create the child
     
     returns: a child chromosome'''

  n = len(x)

  crossover_point = random.randrange(n)

  list_1 = x[0:crossover_point+1]
  list_2 = y[crossover_point+1:n]

  child_chromosome = np.concatenate([list_1, list_2])

     
  return child_chromosome

###Mutation function

In [ ]:
def mutate(child):
  '''take input: a child
     mutates a random 
     gene into another random gene
     
     returns: mutated child'''

  n = len(child)
  random_index = random.randrange(n)
  random_gene = random.randrange(n)

  child[random_index] = random_gene
  

  return child

### Genetic Algorithm Function

In [ ]:
def GA(population, n, mutation_threshold = 0.3):
  '''implement the pseudocode here by
     calling the necessary functions- Fitness, 
     Selection, Crossover and Mutation
     
     print: the max fitness value and the 
     chromosome that generated it which is ultimately 
     the solution board'''

  search_limit = 100000
  goal_fit = (n*(n-1))//2

  counter = search_limit

  print(f"Goal fitness for current population: {goal_fit}.")
  print(f"Population dimension: {n} x {n}.")
  print(f"Population size: {len(population)}.")
  print(f"Search limit: {search_limit}.")

  print("\nRunning....")

  while(counter > 0):
    fitness_scores = fitness(population, n)

    new_population = []

    for i in range(0, len(population)):

      parent_1 = select(population, fitness_scores)
      parent_2 = select(population, fitness_scores)
      child = crossover(parent_1, parent_2)

      if random.uniform(0,1) < mutation_threshold:
        child = mutate(child)

      if findIndividualFitness(child, n) == goal_fit:
        print("....Done")
        print(f"Result: {child} found in {search_limit-counter} generations.")
        return child

      new_population.append(child)

    population = np.copy(new_population)
    counter -= 1


  print(f"No solutions found in {search_limit} generations. Please try again!")

  return None

Running the Genetic Algorithm function

In [ ]:
'''for 8 queen problem, n = 8'''
n = 8

'''start_population denotes how many individuals/chromosomes are there
  in the initial population n = 8'''
start_population = 10 

'''if you want you can set mutation_threshold to a higher value,
   to increase the chances of mutation'''
mutation_threshold = 0.3

'''creating the population with random integers between 0 to 7 inclusive
   for n = 8 queen problem'''
population = np.random.randint(0, n, (start_population, n))

'''calling the GA function'''
result = GA(population, n, mutation_threshold)

Goal fitness for current population: 28.
Population dimension: 8 x 8.
Population size: 10.
Search limit: 100000.

Running....
....Done
Result: [1 4 6 3 0 7 5 2] found in 6983 generations.
